In [ ]:
import re
import time
import json
import pandas as pd
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

from langchain.vectorstores.base import VectorStoreRetriever
from langchain_community.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from geoloc_utils.ragGeocoder import get_coordinates_array, get_distances

from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv

--------------------------------

### Создание ретриверов и определение функций

In [3]:
places_paths = ["../production_data/production_hotels.csv", "../production_data/production_cafes.csv", "../production_data/production_ent.csv"]
docs = []

def docs_cleaning(docs):
    for i in range (len(docs)):
        cleaned_content = re.sub(r'(: [0-9+]\n0:)', ' ', docs[i].page_content).strip()
        docs[i].page_content = cleaned_content
        docs[i].page_content = docs[i].page_content.replace("description: ", "")
    return docs

def create_loader(file_path):
    loader = CSVLoader(file_path=file_path, content_columns=['description'],
        csv_args={
        'delimiter': ',',
        'quotechar': '"',
    })
    docs = loader.load()
    docs = docs_cleaning(docs)
    return docs

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def make_retriver(name: str, documents: list, embedding_generator,text_splitter, search_type: str = "similarity", search_kwargs: dict = {"k": 7}):
    text_splits = text_splitter.split_documents(documents)
    vector_store = Chroma.from_documents(collection_name=name, documents=text_splits, embedding=embedding_generator, persist_directory="/tmp/chroma_db")
    return vector_store.as_retriever(search_type=search_type, search_kwargs=search_kwargs)

hotels_docs = create_loader(places_paths[0])
cafes_docs = create_loader(places_paths[1])
ent_docs = create_loader(places_paths[2])

In [ ]:
# разделяем текст на части
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100, chunk_overlap=50, add_start_index=True
    )
splits = text_splitter.split_documents(docs)

# создаем эмбеддинги для частей текста
embeddings_generator = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    model_kwargs={'device': 'cuda'},
)

'''
db = Chroma(collection_name="cafes")
db._client.delete_collection("cafes")

db = Chroma(collection_name="hotels")
db._client.delete_collection("hotels")

db = Chroma(collection_name="ents")
db._client.delete_collection("ents")
'''

# создаем объект для поиска по векторному хранилищу
hotels_retriever = make_retriver("hotels", hotels_docs, embedding_generator=embeddings_generator, text_splitter=text_splitter)

ent_retriever = make_retriver("ents", ent_docs, embedding_generator=embeddings_generator, text_splitter=text_splitter)

cafes_retriever = make_retriver("cafes", cafes_docs, embedding_generator=embeddings_generator, text_splitter=text_splitter)

### RAG-конвейер

In [5]:
# используемая модель
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature = 0.7,
    api_key=API_KEY,
    base_url=PROXY_URL,
)

# промпты
template_main = """Вы являетесь помощником при составлении маршрутов по Новосибирской области.
Используйте информацию только из найденного контекста, чтобы ответить на вопрос. При вопросе об определенном городе предоставляйте места только из него. Указывайте название, адрес и цену мест. Не добавляйте свои данные. Отвечайте на русском, понятно, кратко и лаконично. Используйте слова из запроса пользователя.
При составлении маршрута на 1 день и более ОБЯЗАТЕЛЬНО указывайте места, где можно покушать и где переночевать. Указывай одно место для ночлега на все дни, если не просят обратного.
Данные о развлечениях Новосибирска: {context_ent}  
Данные об отелях Новосибирска: {context_hotels}
Данные о местах питания в Новосибирске: {context_cafes}
Если вы не знаете ответа или у вас нет данных для корректного ответа на него, просто скажите, что не знаете. 
Вопрос от пользователя: {question} 
"""
prompt = ChatPromptTemplate.from_template(template_main)

template_paths = """Добавь в предложенный маршрут информацию о расстояниях между объектами, которые предложено посетить за день. Если маршрута нет - верни исходный текст. Напишите примерное время в пути при передвижении пешком и на машине. Предупредите, что расстояния и время приблизительные и необходимо уточнить информацию в интернете. В ответе оставь только текст для пользователя.
Предложенный маршрут: {prompt}  
Данные о расстояниях между объектами: {distances}
"""
prompt_paths = ChatPromptTemplate.from_template(template_paths)

In [6]:
def get_retriver_categories(llm, question):
    template = """Есть вопрос от пользователя: {question} 
    # Оцени, будет ли пользователю полезна для ответа на вопрос информация о: местах проживания, местах питания и развлечениях. Чтобы определить надобность, поставь себя на место этого пользователя, не додумывай вопрос пользователя, отвечай четко по тому что спрашивается. Если же просят составить маршрут более чем на 1 день, то отвечай [true, true, true].
    В ответе напиши  список из true и false, где на первом месте будет надобность мест проживания, на втором - мест питания (рестораны, кафе и прочее) и на третьем - развлечения (бассейны, парки и прочее). К примеру, [false, false, true].
    """
    prompt = ChatPromptTemplate.from_template(template)
    result = llm.invoke(prompt.format(question=question))

    try:
        result = result.content
    except AttributeError:
        result = result

    pattern = re.compile(r"\[.*\]")
    match = pattern.search(result)
   # print(match)
    try:
        return json.loads(match[0])
    except:
        return [True,True,True]

In [ ]:
class RawContext:
    def __init__(self, context_name: str = "Unknown", 
                 retriever: VectorStoreRetriever = None, 
                 context_default_output: str = "нет необходимости"):
        self.context_name = context_name
        self.retriever = retriever
        self.context_default_output = context_default_output


def get_contexts_(llm, question: str, raw_contexts: list[RawContext]):
    start_time = time.time()
    categories_bool_list = get_retriver_categories(llm, question)
   # print("Определение необходимых категорий для ретривера: ", time.time()-start_time)

    start_time = time.time()

    result_context = {
        "question": question,
    }
    for raw_context, is_needed in zip(raw_contexts, categories_bool_list):
        if is_needed:
            extracted_context = (raw_context.retriever | format_docs).invoke(question)
        else:
            extracted_context = raw_context.context_default_output
        result_context[raw_context.context_name] = extracted_context
    
  #  print("Определение контекста из ретривера: ", time.time()-start_time)
    return result_context

# Пример вызова rag_chain
def run_rag_chain(question, prompt, prompt_paths):
    # contexts = get_contexts(llm, question)
    context_question = question
    if ("рядом с новос" in question.lower() or "недалеко с новос" in question.lower()):
        context_question = context_question.lower().replace("новосибирск", "")
        context_question += "например, в Бердске, Кольцово, Колывани"
    contexts = get_contexts_(llm, context_question, raw_contexts=
                             [
                                RawContext(context_name="context_hotels", retriever=hotels_retriever),
                                RawContext(context_name="context_cafes", retriever=cafes_retriever),
                                RawContext(context_name="context_ent", retriever=ent_retriever)
                             ])
  #  print(contexts)

    start_time = time.time()
    descr_cafes = []
    descr_hotels = []
    descr_ent = []

    retr_result = contexts

    empty_cat = 0

    for retr in retr_result:
        if retr == 'context_hotels' and retr_result[retr] != 'нет необходимости':
            descr_hotels += retr_result[retr].split(sep='\n\n')
        if retr == 'context_ent' and retr_result[retr] != 'нет необходимости':
            descr_ent += retr_result[retr].split(sep='\n\n')
        if retr == 'context_cafes' and retr_result[retr] != 'нет необходимости':
            descr_cafes += retr_result[retr].split(sep='\n\n')
        if retr_result[retr] == 'нет необходимости':
            empty_cat += 1

    all_descr_df = descr_cafes + descr_hotels + descr_ent
    #print(all_descr_df)
    final_prompt = prompt.format(**contexts)  # Формируем финальный промпт
    response = llm.invoke(final_prompt)  # Вызываем LLM с финальным промптом
   # print("Формирование ответа: ", time.time()-start_time)
    start_time = time.time()
   # print(response.content)
    
    search_text = response.content.lower() + question
    if ("маршрут" not in search_text) and ("план" not in search_text) and ("поездк" not in search_text) and ("путь" not in search_text) and ("пути" not in search_text) and ("путешеств" not in search_text):
        return response.content
    #print("Сейчас напишу примерное расстояние между местами...")
    coord_array = get_coordinates_array(all_descr_df)
   # print("Получение координат для мест: ", time.time()-start_time)
    start_time = time.time()

    distances = get_distances(coord_array)
   # print("Вычисление расстояний между местами: ", time.time()-start_time)
    start_time = time.time()

    path_contexts = dict()
    path_contexts['distances'] = distances
    path_contexts['prompt'] = response.content

    final_prompt = prompt_paths.format(**path_contexts)  # Формируем финальный промпт
    response = llm.invoke(final_prompt)  # Вызываем LLM с финальным промптом

    #contexts['distances'] = distances
    #print(distances)

    try:
        return response.content
    except AttributeError:
        return response

In [11]:
query = "куда сходить в аквапарк"
a = run_rag_chain(query, prompt, prompt_paths)
print(a)

Вы можете посетить аквапарк «Аквамир». Адрес: ул. Яринская, 8. Это самый большой крытый аквапарк в Сибири с 153 аттракционами.


--------------------

## Оценивание

In [24]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    NonLLMContextRecall,
    NonLLMContextPrecisionWithReference
)


eval_llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=API_KEY,
    base_url=PROXY_URL,
)

load_dotenv(find_dotenv())  

False

In [ ]:
# загрузка тестового датасета с вопросами и референсным контекстом
testset_df = pd.read_csv("../eval_utils/testset_only_route.csv", index_col=0)
testset_df["reference_contexts"] = testset_df["reference_contexts"].apply(
    lambda x: None if x == "[]" else eval(x)
)

# генерация ответов на вопросы и  контекста
answers = []
contexts = []

for i in tqdm(range(len(testset_df))):
    answers.append(run_rag_chain(testset_df["question"][i], prompt, prompt_paths))
    contexts.append(
        [
            docs.page_content
            for docs in ent_retriever.invoke(
                testset_df["question"][i])
        ]
        + [
            docs.page_content
            for docs in cafes_retriever.invoke(
                testset_df["question"][i]
            )
        ]
        + [
            docs.page_content
            for docs in hotels_retriever.invoke(
                testset_df["question"][i]
            )
        ]
    )

# формирование датасета
testset_df["answer"] = answers
testset_df["contexts"] = contexts

testset_df.to_csv("../eval_utils/eval_results/production_eval_testset_gpt-4o-mini.csv")

In [ ]:
# подготовка датасета к оценке
from datasets import Dataset

# если импортируем датасет
#testset_df = pd.read_csv('../data/eval_testset_by_quest_gpt4_coord.csv')
#testset_df['reference_contexts'] = testset_df['reference_contexts'].apply(lambda x: None if isinstance(x, float) else eval(x))
#testset_df['contexts'] = testset_df['contexts'].apply(lambda x: None if isinstance(x, float) else eval(x))

testset = Dataset.from_pandas(testset_df)

testset_df_nonan = testset_df[~testset_df.reference_contexts.isna()]
testset_df_nonan = Dataset.from_pandas(testset_df_nonan)

In [30]:
# оценка ретривера
result_context = evaluate(
    dataset = testset_df_nonan,
    llm=eval_llm,
    embeddings=HuggingFaceInferenceAPIEmbeddings(api_key=HG_API_KEY, model_name="sentence-transformers/all-MiniLM-l6-v2"),
    metrics=[
        NonLLMContextRecall(),
        NonLLMContextPrecisionWithReference(),
    ],
    raise_exceptions=False
)
result_context = result_context.to_pandas()

Evaluating: 100%|██████████| 46/46 [00:00<00:00, 172.78it/s]


In [28]:
# оценка модели
result_main = evaluate(
    dataset = testset,
    llm=eval_llm,
    embeddings=HuggingFaceInferenceAPIEmbeddings(api_key=HG_API_KEY, model_name="sentence-transformers/all-MiniLM-l6-v2"),
    metrics=[
        faithfulness,
    ],
    raise_exceptions=False
)
result_main = result_main.to_pandas()

Evaluating: 100%|██████████| 30/30 [03:43<00:00,  7.47s/it]


In [32]:
# подсчет средних значений
mean_faithfulness = result_main['faithfulness'].mean()
mean_context_recall = result_context['non_llm_context_recall'].mean()
mean_context_precision = result_context['non_llm_context_precision_with_reference'].mean()


print("faithfulness: ", mean_faithfulness)
print("context_recall: ", mean_context_recall)
print("context_precision: ", mean_context_precision)

faithfulness:  0.362879292424747
context_recall:  0.7007246376811594
context_precision:  0.3296054305904617


In [ ]:
# подсчет метрик с помощью ИИ
import io
from eval_utils.evalAI import get_mean_metrics

def read_csv_as_text(filepath):
    buffer = io.StringIO()
    pd.read_csv(filepath, usecols=['description'],index_col=0).to_csv(buffer)
    buffer.seek(0)
    return buffer.read()

csv_text = read_csv_as_text(places_paths[0]) + "\n\n" + read_csv_as_text(places_paths[1]) + "\n\n" + read_csv_as_text(places_paths[2])

ai_metrics = get_mean_metrics(eval_llm, csv_text, testset_df)
print(ai_metrics)

100%|██████████| 30/30 [01:29<00:00,  2.99s/it]

{'Достоверность': 5.0, 'Безопасность': 5.0, 'Детальность': 4.966666666666667, 'Релевантность': 4.9, 'Учет цены': 4.7, 'Учет продолжительности': 5.0, 'Полезность': 4.9, 'Логистика': 4.966666666666667}


In [ ]:
from eval_utils.evalAI import get_metrics_for_each_question
coord_metr = get_metrics_for_each_question(eval_llm, csv_text, testset_df)
print(coord_metr)
coord_metr.to_csv("../data/eval_testset_by_quest_gpt4_coord.csv")

100%|██████████| 30/30 [01:28<00:00,  2.96s/it]

                                             question  \
0   Составь маршрут по Новосибирску на 2 дня с пос...   
1   Какой маршрут по Новосибирской области на уике...   
2   Покажи идеи для однодневной поездки в Новосиби...   
3   Нужен маршрут по музеям Новосибирска на 3 дня ...   
4   Составить маршрут по Новосибирску на 1 день с ...   
5   Порекомендуй места для отдыха в Новосибирской ...   
6   Какой маршрут по Новосибирску можно составить ...   
7   Есть ли интересные маршруты на 2 дня по города...   
8   Составь маршрут на 3 дня по природным достопри...   
9   Какой маршрут можно составить по историческим ...   
10  Покажи, где можно провести три дня в Новосибир...   
11  Ищу маршрут по Новосибирску на 2 дня с акценто...   
12  Составь план поездки по культурным местам Ново...   
13  Есть ли план на 5 дней для путешествия по инте...   
14  Какой маршрут на 3 дня по архитектурным памятн...   
15  Порекомендуйте однодневную поездку с активным ...   
16  Составь маршрут по заповедн

----------------------------------------------------------------------------------------------------------------------------------------------------------------